# API Decoder

In [ ]:
from pathlib import Path
import pandas as pd
import gc
import joblib
from tqdm import tqdm
# ------ Custom Imports ------
from sarpyx.processor.core.decode import S1L0Decoder
from sarpyx.utils.io import find_dat_file
# ------ Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
# ------ Setup paths ------
cwd = Path.cwd().parent
data_dir = cwd / 'extracted_data'
output_dir = cwd / 'decoded_data'
# ------ Functions ------
def decode_s1_l0(input_file, output_dir):
    decoder = S1L0Decoder()
    decoder.decode_file(input_file, output_dir, save_to_zarr=True, headers_only=False)
    del decoder
    # Garbage collection
    gc.collect()

def retrieve_input_files(safe_folders, verbose=False):
    """Retrieve input files from SAFE folders."""
    pols = ['vh', 'vv', 'hh', 'hv']
    input_files = []
    folders_map = {x: [] for x in safe_folders}
    for folder in safe_folders:
        for pol in pols:
            try:
                input_file = find_dat_file(folder, pol)
                input_files.append(input_file)
                folders_map[folder].append(input_file)
                if verbose:
                    print(f'📁 Found {input_file.name} in {folder.name}')
            except FileNotFoundError:
                continue
        else:
            if verbose:
                print(f'📁 No .dat file found in {folder.name}, checking next folder...')
            pass
    return input_files, folders_map
            
# Find first SAFE folder and .dat file
safe_folders = [f for f in data_dir.rglob('*.SAFE') if f.is_dir()]
input_files, folders_map = retrieve_input_files(safe_folders)


In [ ]:
# Optional: Dump info about input files and folders
# dump input_files and folders_map to joblib files
joblib.dump(input_files, output_dir / 'info_input_files.joblib')
joblib.dump(folders_map, output_dir / 'info_folders_map.joblib')

# ========== Main Processing ==========
# Process a single file:
single_file = input_files[0]  # Replace 0 with the desired index
if single_file.is_file():
    print(f'🔍 Processing {single_file.name}...')
    decode_s1_l0(single_file, output_dir)
else:
    print(f'❌ {single_file.name} is not a valid file.')

2025-07-11 10:31:03,053 - INFO - Processing file: /Data_large/marine/PythonProjects/SAR/sarpyx/extracted_data/BANGLADESH/S1B_S3_RAW__0SSV_20190202T115525_20190202T115541_014767_01B8AA_2BFE.SAFE/s1b-s3-raw-s-vv-20190202t115525-20190202t115541-014767-01b8aa.dat
2025-07-11 10:31:03,054 - INFO - File size: 417.0 MB
2025-07-11 10:31:03,054 - INFO - Starting full decode process...
2025-07-11 10:31:03,055 - INFO - Starting decode process for: /Data_large/marine/PythonProjects/SAR/sarpyx/extracted_data/BANGLADESH/S1B_S3_RAW__0SSV_20190202T115525_20190202T115541_014767_01B8AA_2BFE.SAFE/s1b-s3-raw-s-vv-20190202t115525-20190202t115541-014767-01b8aa.dat
2025-07-11 10:31:03,054 - INFO - File size: 417.0 MB
2025-07-11 10:31:03,054 - INFO - Starting full decode process...
2025-07-11 10:31:03,055 - INFO - Starting decode process for: /Data_large/marine/PythonProjects/SAR/sarpyx/extracted_data/BANGLADESH/S1B_S3_RAW__0SSV_20190202T115525_20190202T115541_014767_01B8AA_2BFE.SAFE/s1b-s3-raw-s-vv-20190202t1

🔍 Processing s1b-s3-raw-s-vv-20190202t115525-20190202t115541-014767-01b8aa.dat...


decoded: 29812 packets [01:23, 356.86 packets/s]
2025-07-11 10:32:26,601 - INFO - Decoded 29812 records from file
2025-07-11 10:32:26,601 - WARNING - Starting an incomplete sub-commutated data cycle. (first index: 32.
2025-07-11 10:32:26,602 - WARNING - Incomplete sub-commutated data cycle: 0
2025-07-11 10:32:26,604 - WARNING - Incomplete sub-commutated data cycle: 20
2025-07-11 10:32:26,606 - WARNING - Incomplete sub-commutated data cycle: 51
decoded: 29812 packets [01:23, 356.86 packets/s]
2025-07-11 10:32:26,601 - INFO - Decoded 29812 records from file
2025-07-11 10:32:26,601 - WARNING - Starting an incomplete sub-commutated data cycle. (first index: 32.
2025-07-11 10:32:26,602 - WARNING - Incomplete sub-commutated data cycle: 0
2025-07-11 10:32:26,604 - WARNING - Incomplete sub-commutated data cycle: 20
2025-07-11 10:32:26,606 - WARNING - Incomplete sub-commutated data cycle: 51
2025-07-11 10:32:26,609 - WARNING - Incomplete sub-commutated data cycle: 82
2025-07-11 10:32:26,612 - W

Added metadata with 29404 records as zarr attributes


2025-07-11 10:35:46,134 - INFO - Saved unified echo data to Zarr: /Data_large/marine/PythonProjects/SAR/sarpyx/decoded_data/s1b-s3-raw-s-vv-20190202t115525-20190202t115541-014767-01b8aa.zarr
2025-07-11 10:35:46,135 - INFO -   - Echo shape: (29404, 22020)
2025-07-11 10:35:46,136 - INFO -   - Metadata records: 29404
2025-07-11 10:35:46,136 - INFO -   - Ephemeris records: 463
2025-07-11 10:35:46,137 - INFO - Saved burst info: /Data_large/marine/PythonProjects/SAR/sarpyx/decoded_data/s1b-s3-raw-s-vv-20190202t115525-20190202t115541-014767-01b8aa_burst_info.json
2025-07-11 10:35:46,137 - INFO - Saved summary info as JSON: /Data_large/marine/PythonProjects/SAR/sarpyx/decoded_data/s1b-s3-raw-s-vv-20190202t115525-20190202t115541-014767-01b8aa_info.json
2025-07-11 10:35:46,138 - INFO - Created 1 Zarr file(s) and 4 other files
2025-07-11 10:35:46,139 - INFO - 
================================================== ✅ Processed. ==================================================
2025-07-11 10:35:46,135

Added ephemeris with 463 records as zarr attributes
Saved array to /Data_large/marine/PythonProjects/SAR/sarpyx/decoded_data/s1b-s3-raw-s-vv-20190202t115525-20190202t115541-014767-01b8aa.zarr with maximum compression (zstd-9, chunks=(512, 512))


## 🔧 Updated Unified Decoder with Padding Support

The decoder has been updated to handle the case where different bursts have different numbers of samples per line. This is common in SAR data where burst boundaries may not align perfectly.

### Solution: Adaptive Padding
- **Problem**: Burst 0 had 22012 samples, Burst 1 had 22020 samples
- **Solution**: Pad smaller bursts with zeros to match the largest burst width
- **Benefits**: 
  - Preserves all original data
  - Maintains data integrity
  - Enables concatenation into unified structure

### Padding Information Tracking
The decoder now tracks:
- Original width of each burst
- Whether padding was applied
- Amount of padding added
- Final unified width

This ensures full traceability of any modifications made to the original data.